In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import os
import argparse
import numpy as np

import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
import networkx as nx
from matplotlib import pyplot as plt

from minimal.arch import Generator

from minimal.dataset import (
    FloorplanGraphDataset,
    floorplan_collate_fn,
    reader
)

from minimal.utils import (
    init_input,
    draw_masks,
    draw_graph
)

%matplotlib inline

In [20]:
DATA_PATH = "./data/sample_list.txt"

In [21]:
data_path = DATA_PATH
transform = transforms.Normalize(mean=[0.5], std=[0.5])
target_set = 8
split = "test"

In [26]:
subgraphs = []

In [31]:
with open(data_path, "r") as f1:
    lines = f1.readlines()
    lines = list(map(lambda x: x.strip(), lines))

lines

['data/json/45012.json',
 'data/json/45161.json',
 'data/json/36233.json',
 'data/json/19307.json',
 'data/json/7513.json',
 'data/json/18477.json']

In [44]:
for line in lines:
    rms_type, fp_eds, rms_bbs, eds_to_rms, eds_to_rms_tmp = reader(line)
    
    subgraphs.append([
        rms_type,
        rms_bbs,
        fp_eds,
        eds_to_rms,
        eds_to_rms_tmp,
    ])